In [1]:
from scipy.optimize import linprog
import numpy as np

# 4.5.1

In [2]:
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3


$x_{ij}$ - how many items are taken from warehouse i to client j  
$$f = \sum_{i,j} cost_{ij} * x_{ij}$$

In [3]:

c = cost.flatten()
print(c) 

[2 5 3 7 7 6]


For each warehouse number of taken items must be less than stock:
$$\forall i: \sum_j x_{ij} \leq stock_i$$

In [4]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


For each client number of got items must be greater on equal than demand:
$$\forall j: \sum_i x_{ij} \geq demand_j$$
Which is the same as:
$$\forall j: - \sum_i x_{ij} \leq -demand_j$$

In [5]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140]


In [6]:
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: 1900.0
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([110.,   0.,  70.,   0., 150.,  70.])

Answer: 110 items from warehouse 1 to client 1, 110 items from warehouse 1 to client 3,  70 items from warehouse 1 to client 3, 
150 items from warehouse 2 to client 2, 70 items from warehouse 2 to client 3, 

# 4.5.2

In [7]:
import cvxpy as cvx

#### Матрица стоимостей $C$

In [8]:
c = np.array([[1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2], 
    [10, 3, 1000, 6, 20], 
    [19, 7, 6, 1000, 4], 
    [8, 2, 20, 4, 1000]])



#### Матрица переменных $X$

In [9]:
x = cvx.Variable(shape=(5,5), boolean=True)

#### Ограничения (сумма $X$ по строкам и столбцам равна 1)

In [10]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5)
]
    

#### Целевая функция (сумма $CX$)

In [11]:
func = cvx.sum(cvx.multiply(x, c))

In [12]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

#### Ответ

In [13]:
problem.solve()

31.999999999961364

#### Выбранные ячейки матрицы

In [14]:
np.round(x.value)

array([[ 0.,  0., -0., -0.,  1.],
       [ 0., -0.,  0.,  1.,  0.],
       [-0.,  1.,  0., -0., -0.],
       [-0.,  0.,  1.,  0., -0.],
       [ 1.,  0., -0., -0.,  0.]])

# 4.5.3

In [15]:
x = cvx.Variable(shape=(5,5), boolean=True)
u = cvx.Variable(shape=5, integer=True)

In [16]:
from itertools import product

In [17]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5),
    u >= 1,
    u <= 4
]

for i, j in product(range(5), range(5)):
    if i >= 1 and j >= 2 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i,j] <= 4)


In [18]:
func = cvx.sum(cvx.multiply(x, c))

In [19]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

### Ответ

In [20]:
problem.solve()

31.99999999996332

### Выбранные пути

In [21]:
np.round(x.value)

array([[ 0.,  0.,  1., -0., -0.],
       [-0., -0.,  0.,  1.,  0.],
       [-0.,  1., -0.,  0.,  0.],
       [-0., -0.,  0., -0.,  1.],
       [ 1., -0.,  0.,  0., -0.]])

### Значения $u$
$u[0]$ ничего не значит, она не фигурирует ни в ограничениях, ни в целевой функции

In [22]:
np.round(u.value)

array([3., 1., 3., 2., 4.])

Это означает, что вершина A - первая в маршруте, B - третья, C - вторая, D - четвёртая. Эту же информацию можно достать из матрицы $X$.  
Искомое решение.

![alt text](https://i.ibb.co/qJzt4LW/image.png)